In [1]:
import collection.JavaConversions._

import org.apache.hadoop.conf.Configuration
import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.rdd.RDD

import org.bson.BSONObject
import org.bson.types.BasicBSONList
import java.io._
import com.mongodb.hadoop.{
  MongoInputFormat, MongoOutputFormat,
  BSONFileInputFormat, BSONFileOutputFormat}

In [2]:
val mongoConfig = new Configuration()
mongoConfig.set("mongo.input.uri",
    "mongodb://localhost:27017/twitter.tweets")
val documents = sc.newAPIHadoopRDD(
    mongoConfig,                // Configuration
    classOf[MongoInputFormat],  // InputFormat
    classOf[Object],            // Key type
    classOf[BSONObject])        // Value type

In [46]:
val regex = "http://(\\w+.\\w+)/.*".r
val topTags = documents.map{case (a, b) => b}.
  map(_ get("entities")).
  map{case e: BSONObject => e.get("urls")}.
  flatMap{case l: BasicBSONList => l.toMap.map(_._2).toList}.
  flatMap{case b: BSONObject => Some(b.get("expanded_url"))
          case _ => None}.
  flatMap{case s: String => "http://(\\w+.\\w+)/.*".r findFirstMatchIn s}.
  map(_.group(1)).
  countByValue().
  toList.
  sortBy(-_._2)

In [47]:
topTags.take(10)

List((bit.ly,295), (buff.ly,130), (ow.ly,125), (wp.me,46), (goo.gl,25), (firstround.com,22), (snip.ly,21), (dlvr.it,19), (bluehillresearch.com,17), (onforb.es,11))

In [45]:
documents.map{case (a, b) => b}.
  map(_ get("entities")).
  map{case e: BSONObject => e.get("urls")}.
  flatMap{case l: BasicBSONList => l.toMap.map(_._2).toList}.
  flatMap{case b: BSONObject => Some(b.get("expanded_url"))
          case _ => None}.
  flatMap{case s: String => "http://bit.ly/.*".r findFirstIn s}.
  take(5)

Array(http://bit.ly/1PNSyUE, http://bit.ly/1RyUCeo, http://bit.ly/1SjkWi8, http://bit.ly/1XhvNIU, http://bit.ly/1RkWvvv)

In [42]:
documents.map{case (a, b) => b}.
  map(_ get("entities")).
  map{case e: BSONObject => e.get("urls")}.
  map{case l: BasicBSONList => l.get("0")}.
  flatMap{case b: BSONObject => Some(b.get("expanded_url"))
          case _ => None}.
  flatMap{case s: String => "http://buff.ly/.*".r findFirstIn s}.
  take(5)

Array(http://buff.ly/1Poikha, http://buff.ly/1Poikha, http://buff.ly/1Poikha, http://buff.ly/1Poikha, http://buff.ly/1UNUNtq)

In [44]:
documents.map{case (a, b) => b}.
  map(_ get("entities")).
  map{case e: BSONObject => e.get("urls")}.
  map{case l: BasicBSONList => l.get("0")}.
  flatMap{case b: BSONObject => Some(b.get("expanded_url"))
          case _ => None}.
  flatMap{case s: String => "http://ow.ly/.*".r findFirstIn s}.
  take(5)

Array(http://ow.ly/XDQHb, http://ow.ly/104O69, http://ow.ly/104O69, http://ow.ly/104O69, http://ow.ly/YjSSq)